# VGGNet

이정재  
2024-01-30

## \## **VGGNet**

-   VGGNet 구조 살펴보기

<figure class="margin-caption">
<img src="https://miro.medium.com/max/1100/0*6VP81rFoLWp10FcG.png"
alt="VGG" />
<figcaption aria-hidden="true">VGG</figcaption>
</figure>

## `Step 1` : Load libraries & Datasets

In [21]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch import nn

from torchvision import datasets
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_pil_image

# import warnings
# warnings.filterwarnings("ignore")

## `Step 2` : Data preprocessing

불러온 이미지의 증강을 통해 학습 정확도를 향상시키도록 합니다.

`-` RandomCrop  
`-` RandomHorizontalFlip  
`-` Normalize

In [22]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_img = datasets.CIFAR10(
    root = 'data',
    train = True,
    download = True,
    transform = transform,
)

test_img = datasets.CIFAR10(
    root = 'data',
    train = False,
    download = True,
    transform = transform
)

Files already downloaded and verified
Files already downloaded and verified

In [23]:
train_img.data.shape

## `Step 3` : Set hyperparameters

In [24]:
epochs = 10
batch_sizes = 32
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print("Using Device:", device)

Using Device: cuda

## `Step 4` : Create DataLoader

In [25]:
train_loader = DataLoader(train_img, batch_size = batch_sizes, shuffle = True)
test_loader = DataLoader(test_img, batch_size = batch_sizes, shuffle = False)

## `Step 5` : Set Network Structure

In [26]:
# Model
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [27]:
class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 360),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(360, 100),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(100, 10),
        )
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),  # 추가
                           nn.ReLU(inplace=True)]
                in_channels = x
                
        return nn.Sequential(*layers)

## `Step 6` : Create Model instance

In [28]:
model = VGG('VGG16').to(device)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

## `Step 7` : Model compile

In [29]:
# loss
loss = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

## `Step 8` : Set train loop

In [30]:
def train(train_loader, model, loss_fn, optimizer):
    model.train()

    size = len(train_loader.dataset)

    for batch, (X,y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)

        # loss calculation
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}   [{current:>5d}]/{size:5d}')

## `Step 9` : Set test loop

In [31]:
def test(test_loader, model, loss_fn):
    model.eval()

    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y  = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

## `Step 10` : Run model

In [32]:
for i in range(epochs):
    print(f"Epoch {i+1} \n---------------------------")
    train(train_loader, model, loss, optimizer)
    test(test_loader, model, loss)

print("Done!")

Epoch 1 
---------------------------
loss: 2.336054   [    0]/50000
loss: 2.201652   [ 3200]/50000
loss: 1.913308   [ 6400]/50000
loss: 1.833156   [ 9600]/50000
loss: 2.025965   [12800]/50000
loss: 1.458568   [16000]/50000
loss: 1.454589   [19200]/50000
loss: 1.586300   [22400]/50000
loss: 1.777550   [25600]/50000
loss: 1.902973   [28800]/50000
loss: 1.534656   [32000]/50000
loss: 1.808721   [35200]/50000
loss: 1.449842   [38400]/50000
loss: 1.357022   [41600]/50000
loss: 1.378457   [44800]/50000
loss: 1.557457   [48000]/50000
Test Error: 
 Accuracy: 50.6%, Avg loss: 1.338251

Epoch 2 
---------------------------
loss: 1.564054   [    0]/50000
loss: 1.571676   [ 3200]/50000
loss: 1.477239   [ 6400]/50000
loss: 1.520172   [ 9600]/50000
loss: 1.256163   [12800]/50000
loss: 1.066607   [16000]/50000
loss: 1.460073   [19200]/50000
loss: 1.077838   [22400]/50000
loss: 1.215548   [25600]/50000
loss: 0.884830   [28800]/50000
loss: 1.028723   [32000]/50000
loss: 1.288996   [35200]/50000
loss: 1

## \## CIFAR Classifier(Pretrained VGGNet)

ImageNet 데이터로 학습한 VGGNet을 사용하여 주어진 데이터 셋에서 사용할
수 있도록 Fine tuning 해봅니다.